In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 59 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 39.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
# from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, default_data_collator, set_seed
import re
import string


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

loading file https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-large-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-large-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cb

In [ ]:
train = pd.read_csv('/content/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

In [ ]:
train['text_clean'] = train['text'].apply(lambda x: remove_URL(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_emoji(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_html(x))
train['text_clean'] = train['text_clean'].apply(lambda x: remove_punct(x))
train['text_clean'] = train['text_clean'].apply(lambda x: x.lower())
train.head()

,id,keyword,location,text,target,text_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [ ]:
max_length = 64
train['input_ids'] = train['text_clean'].apply(lambda x: tokenizer(x, max_length=max_length, padding="max_length",)['input_ids'])
train['attention_mask'] = train['text_clean'].apply(lambda x: tokenizer(x, max_length=max_length, padding="max_length",)['attention_mask'])
train.rename(columns={'target': 'labels'}, inplace=True)
train.head()

,id,keyword,location,text,labels,text_clean,input_ids,attention_mask
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[101, 2256, 15616, 2024, 1996, 3114, 1997, 202...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[101, 2035, 3901, 2356, 2000, 7713, 1999, 2173...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[101, 19527, 2692, 2111, 4374, 3748, 26332, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[101, 2074, 2288, 2741, 2023, 6302, 2013, 1009...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
train = train[['input_ids', 'attention_mask', 'labels']]
train.head()

,input_ids,attention_mask,labels
0,"[101, 2256, 15616, 2024, 1996, 3114, 1997, 202...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...",1
2,"[101, 2035, 3901, 2356, 2000, 7713, 1999, 2173...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[101, 19527, 2692, 2111, 4374, 3748, 26332, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...",1
4,"[101, 2074, 2288, 2741, 2023, 6302, 2013, 1009...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [ ]:
train_df = train[:-64].reset_index(drop=True)
valid_df = train[-64:].reset_index(drop=True)

In [ ]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 53.1 MB/s 
     |████████████████████████████████| 1.1 MB 27.9 MB/s 
     |████████████████████████████████| 243 kB 47.6 MB/s 
     |████████████████████████████████| 271 kB 46.6 MB/s 
     |████████████████████████████████| 160 kB 44.7 MB/s 
     |████████████████████████████████| 192 kB 52.9 MB/s 


In [ ]:
from datasets import Dataset

In [ ]:
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [ ]:
train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7549
})

In [ ]:
batch_size = 16

args = TrainingArguments(
    'nlp-getting-started',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [ ]:
data_collator = default_data_collator
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
set_seed(42)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 7549
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 8
  Total optimization steps = 177


Epoch,Training Loss,Validation Loss
1,No log,0.187162
2,No log,0.155791
3,No log,0.122557


***** Running Evaluation *****
  Num examples = 64
  Batch size = 16
Saving model checkpoint to nlp-getting-started/checkpoint-59
Configuration saved in nlp-getting-started/checkpoint-59/config.json
Model weights saved in nlp-getting-started/checkpoint-59/pytorch_model.bin
tokenizer config file saved in nlp-getting-started/checkpoint-59/tokenizer_config.json
Special tokens file saved in nlp-getting-started/checkpoint-59/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 64
  Batch size = 16
Saving model checkpoint to nlp-getting-started/checkpoint-118
Configuration saved in nlp-getting-started/checkpoint-118/config.json
Model weights saved in nlp-getting-started/checkpoint-118/pytorch_model.bin
tokenizer config file saved in nlp-getting-started/checkpoint-118/tokenizer_config.json
Special tokens file saved in nlp-getting-started/checkpoint-118/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 64
  Batch size = 16
Saving model checkpoint to nlp

TrainOutput(global_step=177, training_loss=0.37952664477676995, metrics={'train_runtime': 1829.397, 'train_samples_per_second': 12.379, 'train_steps_per_second': 0.097, 'total_flos': 2638181198161152.0, 'train_loss': 0.37952664477676995, 'epoch': 3.0})

In [ ]:
test = pd.read_csv('/content/test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
test['text_clean'] = test['text'].apply(lambda x: remove_URL(x))
test['text_clean'] = test['text_clean'].apply(lambda x: remove_emoji(x))
test['text_clean'] = test['text_clean'].apply(lambda x: remove_html(x))
test['text_clean'] = test['text_clean'].apply(lambda x: remove_punct(x))
test['text_clean'] = test['text_clean'].apply(lambda x: x.lower())
test.head()


,id,keyword,location,text,text_clean
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills 28 in china and taiwan


In [ ]:
test['input_ids'] = test['text_clean'].apply(lambda x: tokenizer(x, max_length=max_length, padding="max_length",)['input_ids'])
test['attention_mask'] = test['text_clean'].apply(lambda x: tokenizer(x, max_length=max_length, padding="max_length",)['attention_mask'])
test.head()

,id,keyword,location,text,text_clean,input_ids,attention_mask
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash,"[101, 2074, 3047, 1037, 6659, 2482, 5823, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,"[101, 2657, 2055, 8372, 2003, 2367, 3655, 2994...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,"[101, 2045, 2003, 1037, 3224, 2543, 2012, 3962...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,"[101, 16976, 7497, 21878, 3748, 26332, 102, 0,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills 28 in china and taiwan,"[101, 15393, 2061, 12672, 10626, 8563, 2654, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [ ]:
test = test[['input_ids', 'attention_mask']]
test.head()

,input_ids,attention_mask
0,"[101, 2074, 3047, 1037, 6659, 2482, 5823, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[101, 2657, 2055, 8372, 2003, 2367, 3655, 2994...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
2,"[101, 2045, 2003, 1037, 3224, 2543, 2012, 3962...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[101, 16976, 7497, 21878, 3748, 26332, 102, 0,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[101, 15393, 2061, 12672, 10626, 8563, 2654, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [ ]:
test_ds = Dataset.from_pandas(test)
test_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3263
})

In [ ]:
outputs = trainer.predict(test_ds)

***** Running Prediction *****
  Num examples = 3263
  Batch size = 16
